# Statystyka w Analizie Danych

## Laboratorium 5 - algorytm normalizacja, selekcja cech.


### Opis
Celem laboratorium jest przeprowadzenie normalizacji i selekcji cech.


### Zbiór danych

Zbiór danych znajduje się w katalogu `dataset/*`. Jest to zmodyfikowany zbiór danych znajdujący się pod adresem: <https://archive.ics.uci.edu/ml/datasets/leaf>.

### Przesyłanie zadań

Wszystkie pliki należy spakować archiwizatorem **zip** i przesłać za pośrednictwem platformy WIKAMP. Poniżej oczekiwana zawartość archiwum:

```
+-- 📂 [IMIE.NAZWISKO].zip
    +-- 📜 Lab05.ipynb
    +-- 📂 dataset
        +-- 📜 dataset.npz
        +-- 📜 ReadMe.pdf
```

**Pamiętaj, wyniki powinny być czytelnie opisane oraz zaprezentowane graficznie (jeżeli jest taka możliwość).**

Przykład (na podstawie tablicy pomyłek):

**Źle** (nie wiadomo co jest poniżej zaprezentowane, kolumny ani wiersze nie są podpisane, nie wiadomo które z nich prezentują predykcje, a które właściwe etykiety):
```
array([[2, 0, 0],
       [0, 0, 1],
       [1, 0, 2]])
```

### Zadanie

Należy wykonać następujące czynności w celu realizacji niniejszego zadania:

#### Normalizacja
* Wczytaj dane.
* Znormalizuj dane.
* Przeprowadź eksperyment z zastosowaniem algorytmu kNN lub NM dla danych znormalizowanych oraz bez normalizacji.
    * W eksperymencie wybierz 5 klas oraz 10 cech.
* Przedstaw porównanie wyników klasyfikacji na danych znormalizowanych i bez normalizacji.
* Napisz wnioski.

**UWAGA: Wykorzystaj gotową implementację kNN [KNeighborsClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)**

In [1]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from matplotlib.axes._axes import _log as matplotlib_axes_logger
from sklearn.feature_selection import SelectKBest, chi2


matplotlib_axes_logger.setLevel('ERROR')


with open('dataset.npz', 'rb') as f:
    data = np.load(f)
    train, test = data['train'], data['test']

In [2]:
def normalize(arr):
    return ((arr - arr.min())/(arr.max()-arr.min()))

In [3]:
def edist(p1, p2, attributes):
    p1_new = []
    p2_new = []
    distance = 0
    
    for i in attributes:
        p1_new.append(p1[i])
        p2_new.append(p2[i])
    
    for i in range(len(p1_new)):
        distance += (p1_new[i] - p2_new[i])**2
        
    return distance**0.5

In [4]:
def average(lst):
    return sum(lst)/len(lst)

In [5]:
def get_mean(train, leaf_classes):
    leaves_mean =[]
    for c in leaf_classes:
        leafLst = train[train[:,0] == c]
        oneAtribute = leafLst[:, 0]
        avgOneLeaf = []
        for i in range(len(leafLst[0])):
            oneAtribute = leafLst[:, i]
            mean = average(oneAtribute)
            avgOneLeaf.append(mean)
        leaves_mean.append(avgOneLeaf)
    return leaves_mean

In [6]:
def chooseClass(classes):
    for i in range(len(classes)):
        if i == 0:
            new_train = np.array(train[train[:,0] == classes[i]])
            new_test = np.array(test[test[:,0] == classes[i]])
        else:
            new_train = np.vstack((new_train,train[train[:,0] == classes[i]]))
            new_test = np.vstack((new_test,test[test[:,0] == classes[i]]))
    return new_train, new_test

In [7]:
def NM(test, attributes, means):
    classified = np.copy(test)
    for te in range(len(test)):
        nearest = 100
        for i in range(len(means)):
            distance = edist(test[te], means[i], attributes)
            if distance < nearest:
                nearest = distance
                classified[te][0] = means[i][0] 
    return classified

In [8]:
def acc(classified,test):
    correct = (test[:,0] == classified[:,0])
    correct = np.count_nonzero(correct)
    return correct/len(classified)

# Wyniki

In [19]:
classes=[3,5,6,7,4]

newTrain,newTest = chooseClass(classes)
normalizeTrain = np.copy(newTrain)
normalizeTest = np.copy(newTest)

# Bez normalizacji
xTrain = newTrain[:,2:]
xTest = newTest[:,2:]
yTrain = newTrain[:,0]
yTest = newTest[:,0]

# Normalizacja
for i in range(2,16):
    toNormalize = normalizeTrain[:,i]
    normalizeTrain[:,i] = normalize(toNormalize)
       
for i in range(2,16):
    toNormalize = normalizeTest[:,i]
    normalizeTest[:,i] = normalize(toNormalize)

xTrainNormalize = normalizeTrain[:,2:]
xTestNormalize = normalizeTest[:,2:]
yTrainNormalize = normalizeTrain[:,0]
yTestNormalize = normalizeTest[:,0]

# NM

In [20]:
attributes = [2,3,4,5,6,7,8,9,10,11]

means = get_mean(newTrain, classes)
classified = NM(newTest,attributes,means)
accuracy = acc(classified, newTest)
print('Accuracy bez normalizacji: ', accuracy)

means = get_mean(normalizeTrain, classes)
classified = NM(normalizeTest,attributes,means)
accuracy = acc(classified, normalizeTest)
print('Accuracy z normalizacją: ', accuracy)

Accuracy bez normalizacji:  0.7851239669421488
Accuracy z normalizacją:  0.8925619834710744


# kNN

In [27]:
model = KNeighborsClassifier(n_neighbors=3)
model.fit(xTrain[:,2:12], yTrain)
print('Bez normalizacji 10 atrybutów:', model.score(xTest[:,2:12],yTest))

model = KNeighborsClassifier(n_neighbors=3)
model.fit(xTrainNormalize[:,2:12], yTrainNormalize)
print('Po normalizacji 10 atrybutów: ', model.score(xTestNormalize[:,2:12],yTestNormalize))

Bez normalizacji 10 atrybutów: 0.8347107438016529
Po normalizacji 10 atrybutów:  0.7768595041322314


#### Selekcja cech
Na tym samym podzbiorze danych (co w poprzednim zadaniu).
* Przeprowadź selekcję cech (wybierz {2, 5} cech) za pomocą metod poznanych na wykładzie (np. z zastosowaniem współczynnika Fishera) lub istniejących implementacji z biblioteki [scikit-learn](https://scikit-learn.org/stable/modules/feature_selection.html) (np. [SelectKBest](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest)).
* Przeprowadź klasyfikację na wybranych cechach.
* Porównaj wyniki klasyfikacji dla różnej liczby cech:
    * 10 cech bez normalizacji (wyniki z poprzedniego zadania),
    * 10 cech z normalizacją (wyniki z poprzedniego zadania),
    * 5 wybranych cech z bez normalizacji,
    * 5 wybranych cech z normalizacją,
    * 2 wybranych cech z bez normalizacji,
    * 2 wybranych cech z normalizacją.
* Opisz wyniki i napisz wnioski.


In [32]:
model5 = SelectKBest(k=5)
xTrain5 = model5.fit_transform(xTrain, yTrain)
xTest5 = model5.transform(xTest)

knModel5 = KNeighborsClassifier(n_neighbors=3)
knModel5.fit(xTrain5, yTrain)

print('Bez normalizacji 5 atrybutów: ',knModel5.score(xTest5, yTest))

modelN5 = SelectKBest(k=5)
xTrainNorm5 = modelN5.fit_transform(xTrainNormalize, yTrainNormalize)
xTestNorm5 = modelN5.transform(xTestNormalize)

knModelN5 = KNeighborsClassifier(n_neighbors=3)
knModelN5.fit(xTrainNorm5, yTrainNormalize)

print('Po normalizacji 5 atrybutów: ',knModelN5.score(xTestNorm5,yTestNormalize))

Bez normalizacji 5 atrybutów:  0.8181818181818182
Po normalizacji 5 atrybutów:  0.8471074380165289


In [33]:
model2 = SelectKBest(k=2)
xTrain2 = model2.fit_transform(xTrain, yTrain)
xTest2 = model2.transform(xTest)

knModel2 = KNeighborsClassifier(n_neighbors=3)
knModel2.fit(xTrain2, yTrain)

print('Bez normalizacji 2 atrybutów: ',knModel2.score(xTest2, yTest))

modelN2 = SelectKBest(k=2)
xTrainNorm2 = modelN2.fit_transform(xTrainNormalize, yTrainNormalize)
xTestNorm2 = modelN2.transform(xTestNormalize)

knModelN2 = KNeighborsClassifier(n_neighbors=3)
knModelN2.fit(xTrainNorm2, yTrainNormalize)

print('Po normalizacji 2 atrybutów: ',knModelN2.score(xTestNorm2,yTestNormalize))

Bez normalizacji 2 atrybutów:  0.6611570247933884
Po normalizacji 2 atrybutów:  0.6322314049586777


Normalizacja raz działa raz nie dziąła. Gdy sprawdząłem dla różnych klas liści wyniki potrafiły się zmieniać na korzyść. Dla 5 atrybutów normalizacja pomogła jednak dla 2 delikatnie pogorszyła wyniki 